# Embedding Model with Instructions

## Datasets

In [29]:
def display_dataset_info(dataset):
    info = dataset.info
    dataset_name = info.dataset_name
    splits_info = info.splits
    features = info.features
    print(f"Dataset Name: {dataset_name}")
    print("Splits Info:")
    for split_name, split_info in splits_info.items():
        num_examples = split_info.num_examples
        print(f" - Split: {split_name}, Num Examples: {num_examples}")
    print("Features:")
    for feature_name, feature_info in features.items():
        print(f" - {feature_name}: {feature_info}")

In [30]:
from datasets import load_dataset

def get_dataset(dataset_name, train_size=0, test_size=0):  
    # Load the dataset
    dataset = load_dataset(dataset_name)
    #display_dataset_info(dataset['test'])
    
    # Access the train, test splits
    train_dataset = dataset['train']
    test_dataset = dataset['test']

    # Random sample the dataset, only use random_sample_size
    if(train_size != 0):
        train_dataset = train_dataset.shuffle(seed=42).select(range(train_size))
    if(test_size != 0):
        test_dataset = test_dataset.shuffle(seed=42).select(range(test_size))

    return train_dataset, test_dataset

## Embedding Models

In [31]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import warnings

def load_model(model_name):
    warnings.filterwarnings("ignore")
    match model_name:
        case "Bert1":
            return pipeline("feature-extraction", model="google-bert/bert-base-uncased", device=0)
        case "Bert2":
            return pipeline("feature-extraction", model="google-bert/bert-large-uncased", device=0)
        case "Instructor":
            return SentenceTransformer("hkunlp/instructor-large")
        case _:
            return None

### BERT - Pipeline

In [32]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np

def encode_Bert(model, dataset, key="text", truncation=True, padding=True, max_length=512, use_cls=True):
    data = KeyDataset(dataset, key)
    pipe = model(data, return_tensors=True, truncation=truncation, padding=padding, max_length=max_length)
    embeddings=[]
    for tensor in tqdm(pipe, desc="Encoding"): 
        if use_cls:
            # Shape [batch_size, sequence_length, hidden_size]
            embedding = tensor[0, 0, :].detach().numpy()
            print(embedding.shape)
        else:
            embedding = tensor.mean(dim=1).flatten()
        embeddings.append(embedding)
    return np.array(embeddings), np.array(dataset["label"])

### Instructor - Sentence Transformers

In [33]:
def encode_ST(model, dataset):
    embeddings = []
    texts = dataset["text"]
    instructions = dataset["instruction"]
    for text, instruction in tqdm(zip(texts, instructions),total=len(dataset), desc="Encoding"):
        embedding = model.encode([[instruction, text]])[0]
        embeddings.append(embedding)
    return np.array(embeddings), np.array(dataset["label"])

## Augment Instruction

In [34]:
def mapper_affixes(example, prefix, suffix):
    example['text'] = prefix + example['text'] + suffix
    return example

def mapper_instruct(example, instruction):
    example['instruction'] = instruction
    return example

def augment_dataset_Affix(dataset, prefix, suffix):
    augmented_dataset = dataset.map(lambda x: mapper_affixes(x, prefix, suffix))
    return augmented_dataset

def augment_dataset_Inst(dataset, instruction):
    augmented_dataset = dataset.map(lambda x: mapper_instruct(x, instruction))
    return augmented_dataset

## Evaluation

In [35]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

def evaluate(method, train_embeddings, test_embeddings, train_labels, test_labels):
    if method == "SVM":
        model = SVC(kernel='linear')
        
    elif method == "MLP":
        model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                          solver='sgd', verbose=1, random_state=1,
                          learning_rate_init=.1)

    model.fit(train_embeddings, train_labels)
    predicted_labels = model.predict(test_embeddings)
    print("Report on " + method + ": ")
    print(classification_report(y_true = test_labels, y_pred = predicted_labels, digits=4))

## EmbedFlow

In [36]:
def EmbedFlow_Bert_1(dataset_name, train_size, test_size, evaluator, prefix, suffix):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)

    # Load Model
    model = pipeline("feature-extraction", model="google-bert/bert-base-uncased", device=0)
    
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Bert(model, train_dataset, use_cls=False)
    test_embeddings, test_labels = encode_Bert(model, test_dataset, use_cls=False)

    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

In [37]:
def EmbedFlow_Bert_2(dataset_name, train_size, test_size, evaluator, prefix, suffix):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)

    # Load Model
    model = pipeline("feature-extraction", model="google-bert/bert-large-uncased", device=0)
    
    train_dataset = augment_dataset_Affix(train_dataset, prefix, suffix)
    test_dataset = augment_dataset_Affix(test_dataset, prefix, suffix)

    # Embed Dataset
    train_embeddings, train_labels = encode_Bert(model, train_dataset, use_cls=False)
    test_embeddings, test_labels = encode_Bert(model, test_dataset, use_cls=False)

    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

In [38]:
def EmbedFlow_Instructor(dataset_name, train_size, test_size, evaluator, instruction):
    # Load Dataset
    train_dataset, test_dataset = get_dataset(dataset_name, train_size, test_size)

    # Load Model
    model = SentenceTransformer("hkunlp/instructor-large")
    
    # Add Instruction
    train_dataset = augment_dataset_Inst(train_dataset, instruction)
    test_dataset = augment_dataset_Inst(test_dataset, instruction)

    # Embed Dataset
    train_embeddings, train_labels = encode_ST(model, train_dataset)
    test_embeddings, test_labels = encode_ST(model, test_dataset)
    
    # Evaluate
    evaluate(evaluator, train_embeddings, test_embeddings, train_labels, test_labels)

## Testing
[Google Sheet](https://docs.google.com/spreadsheets/d/1iBDq7C59G6olf_of_sTF5oCY3Itj6_kImzeUl3XMpd8/edit#gid=1587051763)

In [39]:
import warnings
warnings.filterwarnings("ignore")

datasets = ['stanfordnlp/imdb', 
            'yelp_review_full',
            'Voice49/arXiv-Abstract-Label-20k']

evaluator = ['SVM', 'MLP']

prefix = ['',
          'Movie Review: ', 
          'Restaurant Review: ', 
          'Sentiment Analysis: ', 
          'User Feedback: ', 
          'Customer Experience: ',
          'Product Review: ',
          'Service Feedback: ',
          'Experience at: ',
          'Abstract: ',
          'Research Paper Abstract:',
          'Paper Summary: ']
suffix = ['','']

train_size = 1000
test_size  = 1000

# EmbedFlow_Bert_1(datasets[2], train_size, test_size, evaluator[0], prefix[0], suffix[0])

for dataset in datasets:
        for instruction in prefix:
            print(f"Processing dataset: {dataset}, instruction: '{instruction}'")
            # EmbedFlow_Bert_1(dataset, train_size, test_size,evaluator[0], instruction, '')
            EmbedFlow_Instructor(dataset, test_size, test_size, evaluator[0], instruction)

Processing dataset: stanfordnlp/imdb, instruction: ''


c:\Users\Voice\anaconda3\envs\embed\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Report on SVM: 
              precision    recall  f1-score   support

           0     0.9556    0.9258    0.9405       512
           1     0.9246    0.9549    0.9395       488

    accuracy                         0.9400      1000
   macro avg     0.9401    0.9403    0.9400      1000
weighted avg     0.9405    0.9400    0.9400      1000

Processing dataset: stanfordnlp/imdb, instruction: 'Movie Review: '


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Report on SVM: 
              precision    recall  f1-score   support

        cs.*     0.9412    0.7869    0.8571       122
      econ.*     0.5769    0.2479    0.3468       121
      eess.*     0.8797    0.9141    0.8966       128
      math.*     0.9837    0.9758    0.9798       124
   physics.*     0.8333    0.9091    0.8696       132
     q-bio.*     0.9123    0.8387    0.8739       124
     q-fin.*     0.9291    0.8551    0.8906       138
      stat.*     0.4780    0.8829    0.6203       111

    accuracy                         0.8040      1000
   macro avg     0.8168    0.8013    0.7918      1000
weighted avg     0.8236    0.8040    0.7977      1000

